In [9]:
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
# Ensure the necessary libraries are installed

In [ ]:
def inside_product_page(driver, product_url):
    

In [ ]:
def get_items(driver):
    products = {}
    try:
        # Wait for the search results to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "pt__link"))
        )
    except TimeoutException:
        print("Search results did not load in time.")
        return products

    product_elements = driver.find_elements(By.CLASS_NAME, "pt")

    for indx, product in enumerate(product_elements):
        try:
            product_name = product.find_element(By.CLASS_NAME, "pt__link").get_attribute("title")
            product_price = product.find_element(By.CLASS_NAME, "pt__cost__retail-price").text

            products[product_name] = product_price

        except NoSuchElementException as e:
            print(f"Error retrieving product details for index {indx}: {e}")
            continue

    return products  # <-- moved outside the for loop

In [10]:
def reject_cookies(driver):
    try:
        # Wait for the cookie banner to appear and reject cookies - CAN TRY DIFFERENT WAY OF LOCATING HTMLS
        WebDriverWait(driver, 7).until(
            EC.presence_of_element_located((By.ID, "onetrust-reject-all-handler")) # wait for the cookie bannr
            ).click()
    except (TimeoutException, NoSuchElementException):
        print("Cookie banner not found or already rejected.")

In [45]:
def element_exists(driver, by, value, timeout=5):
    try:
        WebDriverWait(driver, timeout).until(EC.presence_of_element_located((by, value)))
        return True
    except TimeoutException:
        return False

try to search by id

In [ ]:
url =  "https://www.sainsburys.co.uk"

driver = webdriver.Chrome()
driver.get(url)

reject_cookies(driver)
search = driver.find_element(By.ID, "term")
search.clear()  # Clear the search field if needed

for char in "milk":
    search.send_keys(char)  # Type each character into the search field
    time.sleep(0.5)  # Wait for half a second between keystrokes
search.send_keys(Keys.RETURN)   # Press Enter to submit the search

all_items = {}

if element_exists(driver, By.CLASS_NAME, "conditional-render-wrapper"):

    # Loop while the next button exists and is not disabled
    while True:
        all_items.update(get_items(driver))
        next_buttons = driver.find_elements(By.CSS_SELECTOR, "[rel='next']")
        if not next_buttons:
            break
        next_button = next_buttons[0]
        # Check if the next button has the 'is-disabled' class
        if 'is-disabled' in next_button.get_attribute('class'):
            break
        next_button.click()
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "pt"))
        )

else:
    all_items.update(get_items(driver))

df = pd.DataFrame(all_items.items(), columns=['Product', 'Price'])

df  # display the DataFrame

,Product,Price
0,Cravendale Filtered Fresh Semi Skimmed Milk 2L...,£2.90
1,Riddles Mango Ice Tea Blended with Rum 250ml,£2.20
2,Weetabix Crispy Minis Chocolate Chip 500g,£3.80
3,Purina One Adult Dry Cat Food Salmon & Wholegr...,£6.00
4,"Sainsbury's Chilli Con Carne, Be Good To Yours...",£3.50


In [16]:
import sys
print(sys.executable)

/Users/siddharthgianchandani/Desktop/vscode projects/consumption-gaps/.venv/bin/python
